# Import and setup

In [ ]:
!which python

In [ ]:
import os
import sys
difsr_root= os.path.dirname(os.getcwd())
sys.path.insert(1, difsr_root)
sys.path

In [ ]:
import os 
import pandas as pd
import numpy as np
from datetime import datetime
from recbole.config import Config
from recbole.data import create_dataset
from recbole.data.utils import get_dataloader
from recbole.utils import init_logger, init_seed, get_model, get_trainer, set_color
import matplotlib.pyplot as plt

In [ ]:
dataset_root = os.path.abspath('../dataset')
# os.listdir(dataset_root)
os.listdir('../dataset/')

In [ ]:
from dataclasses import dataclass

@dataclass
class Arguments:
    model:str = 'SASRecD'
    dataset:str = 'yelp'
    config_files:str = None





# Yelp

In [ ]:
dataset_path=os.path.join(dataset_root,'yelp')
os.listdir(dataset_path)

## EDA

### Reading the dataset using recbole

In [ ]:
# reading the dataset through the recbole framework
args=Arguments(dataset="yelp",config_files=os.path.join(difsr_root,'configs/yelp_cat_L1.yaml'))
config_file_list = args.config_files.strip().split(' ') if args.config_files else None
config = Config(model=args.model, dataset=f'{args.dataset}', config_file_list=config_file_list)
config.final_config_dict['data_path'] = os.path.join(difsr_root,config.final_config_dict['data_path'])
config.final_config_dict

In [ ]:
dataset = create_dataset(config)
dataset

In [ ]:
dataset.item_feat.head()

In [ ]:
# print the list of columns that can be considered as features
dataset.field2token_id.keys()

In [ ]:
dataset.item_feat['categories'].loc[0].dtype

### read item data

In [ ]:
ydf=pd.read_csv(os.path.join(dataset_path,'yelp.item'),sep='\t')
ydf.head()

In [ ]:
# check how many non-Nan values are there in each column
ydf.count()/len(ydf)

### read original data
reading the data from the Yelp dataset, before it has been preprocessed by recbole. the dataset was downloaded from Kaggle (version 4)


In [ ]:
import json
yelp_orig_path='/home/guy/sd1tb/datasets/yelp'
os.listdir(yelp_orig_path)


In [ ]:
#read json file to dataframe
def read_json_to_df(path):
    with open(path) as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame.from_dict(data)

yelp_review=read_json_to_df(os.path.join(yelp_orig_path,'yelp_academic_dataset_review.json'))
yelp_review.head()

In [ ]:
# group by reviews of user u by day of review and count the number of reviews
u.groupby(u['date'].dt.date).count()['review_id']

In [ ]:
# find how many users have more than 100 reviews
len(yelp_review.groupby('user_id').count()['review_id'][yelp_review.groupby('user_id').count()['review_id']>100])

In [ ]:
yelp_review['user_id'].value_counts()

## running predictions

In [ ]:
import torch



In [ ]:
# reading the dataset through the recbole framework
args=Arguments(dataset="yelp",config_files=os.path.join(difsr_root,'configs/yelp_cat_L1.yaml'))
config_file_list = args.config_files.strip().split(' ') if args.config_files else None
config = Config(model=args.model, dataset=f'{args.dataset}', config_file_list=config_file_list)
config.final_config_dict['data_path'] = os.path.join(difsr_root,config.final_config_dict['data_path'])
config.final_config_dict

In [ ]:
ITEM_ID = config['ITEM_ID_FIELD']
ITEM_SEQ = ITEM_ID + config['LIST_SUFFIX']
ITEM_SEQ_LEN = config['ITEM_LIST_LENGTH_FIELD']

In [ ]:
# create a dataset of type SequentialDataset
dataset = create_dataset(config)
dataset

In [ ]:
built_datasets = dataset.build()
train_dataset, valid_dataset, test_dataset = built_datasets
test_dataset

In [ ]:
idx=0
n_inter=1 # number of interactions to sample
interaction = test_dataset[idx:idx+n_inter]
pos_i = interaction[ITEM_ID]
interaction

In [ ]:
#create an instance of the model
model = get_model(config['model'])(config, dataset).to(config['device'])
model 

In [ ]:
# set model path
model_path=os.path.join(difsr_root,'saved/yelp_cat_L1')
os.listdir(model_path)


In [ ]:
model_file_name=os.path.join(model_path,'SASRecD-Apr-09-2023_20-03-18.pth')
# load the model
model_file = torch.load(model_file_name)
model.load_state_dict(model_file['state_dict'])
model.load_other_parameter(model_file.get('other_parameter'))
print('Loading model structure and parameters from {}'.format(model_file))
model.eval()


In [ ]:
# to get the score of the prediction (the cosine similarity between the embedding of the predicted item and the embedding of the ground truth item)
scores = model.predict(interaction.to(config['device']))
scores

In [ ]:
# to get the embedding of the predicted item
item_seq = interaction[ITEM_SEQ].to(config['device'])
item_seq_len = interaction[ITEM_SEQ_LEN].to(config['device'])
gt_item = interaction[ITEM_ID]
item_embeddings = model.item_embedding.weight
seq_output=model(item_seq, item_seq_len)
scores = torch.matmul(seq_output, item_embeddings.transpose(0, 1))  # [B, item_num]

In [ ]:
topk=100
topk_scores, topk_items = torch.topk(scores, topk)
print(topk_items)


In [ ]:
# check if the ground truth item is in the topk items
gt_item in topk_items.detach().cpu().numpy()